In [73]:
#read in
using JuMP, Cbc
m = Model(solver=CbcSolver(logLevel=1))


Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver set to Cbc

In [74]:
#hindex = readcsv("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/hindex.dat")

hindex = 1:24#readcsv("hindex.dat")
#currently, 1680 runs at 6 sec with this gen asset mix: [0 0 6 3 0 0]
#24*2 = 2 sec with gen asset mix: [0 144.9 6 3 189 6.93]
#24*3 = 1 sec
#24*4 = 3.4 sec
#24*5 = 2.3 sec
#24*6 = 2.4 
#24*7 = 6.4
#24*8 = 161 + (stopped by force)

T = length(hindex) #Prediction Horizon in hours

@defVar(m, C[t=1:T] >= 0) #Cost of diesel
@defVar(m, Cs[t=1:T] >= 0) #Cost of start up generator
@defVar(m, Pus[t=1:T] >= 0) #Unserved Power
@defVar(m, Plost[t=1:T] >= 0) #spilled power
@defVar(m, SOC[t=1:T] >= 0) # SOC of battery
@defVar(m, SOH[t=1:T] >= 0) #State of health of battery (degradation costs)
@defVar(m, Pbc[t=1:T] >= 0) #battery charging power
@defVar(m, Pbd[t=1:T] >= 0) #battery discharging power 
@defVar(m, Pl[t=1:T] >= 0) #demand load 
@defVar(m, Pd[t=1:T] >= 0) #diesel power 
@defVar(m, q[t=1:T] >= 0) # quantity of diesel consumption
@defVar(m, Bbc[t=1:T], Bin) #batt charge
@defVar(m, Bbd[t=1:T], Bin) #batt discharge

#define these for critical and noncritical
@defVar(m, PusC[t=1:T] >= 0)
@defVar(m, PusNC[t=1:T] >= 0)

for t=1:T
    @addConstraint(m, PusC[t] + PusNC[t] == Pus[t]) 
end 


In [75]:
#genAssets = readcsv("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/genAssets.dat")
#genAssets = readcsv("genAssets.dat")
#genAssets = [0 0 6 3 0 0]
#genAssets = [50 0 0 3 0 0]
genAssets = [0 144.9 6 3 189 6.93]

# [PVbank BattNomE Gensetsize dieselcost battdischarge battchargeP]

PVbank = genAssets[1] #this is the total capacity of the PV panels -- needs to be read in from REM 
delta = 1 #duration of time period (hr)
#Cus = 5 #cost of Non-served E ($/KWH)
Clost = 0 #cost of spillage ($/KWH)
Cb = .1 #battery degradation cost ($)
Cd = 1 #fixed start up cost ($)
Pb = genAssets[5] #battery discharge power (KW)
EFFd = 1 #Efficiency of battery discharge
EFFc = 1 #Efficiency of battery charge
Eb = genAssets[2] #battery nominal energy (KWH)
Xmin = 1 #min generated diesel power(KW) -- Feed this in?
Xmax = genAssets[3] #max generated diesel power(KW) -- Feed this in?
Cc = genAssets[4] #cost of diesel ($/L)
Pb_dc = genAssets[6] #Battery charge power

# We may not need a cost for spillage of solar 

CusC = 1 #cost of unable to supply critical
CusNC = 0.33 #cost of unable to supply noncritical


0.33

In [76]:
#Initial Variables 

#PlC = readcsv("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/critical_demand_Output.dat")
PlC = readcsv("critical_demand_Output.dat")
#PlC = ones(T)*15
PlC = PlC[hindex]

#PlNC = readcsv("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/Ncritical_demand_Output.dat")
PlNC = readcsv("Ncritical_demand_Output.dat")
#PlNC = ones(T)*10
PlNC = PlNC[hindex]
#Pl = PlC + PlNC

#Ppv = readcsv("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/PV_Output.dat")
Ppv = readcsv("PV_Output.dat")
#Ppv = ones(T)*5
Ppv = Ppv[hindex] #these values are for per KW systems so should multiply by PV capacity 


24-element Array{Float64,1}:
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     
 0.004878
 0.052358
 0.13235 
 0.30173 
 0.58855 
 0.63881 
 0.67861 
 0.53453 
 0.62649 
 0.42951 
 0.32497 
 0.081879
 0.032782
 0.0     
 0.0     
 0.0     
 0.0     
 0.0     

In [77]:
@addConstraint(m, sum{PusC[t] + PusNC[t], t=1:T} <= 0.6*sum{PlC[t] +PlNC[t],t=1:T} )

PusC[1] + PusNC[1] + PusC[2] + PusNC[2] + PusC[3] + PusNC[3] + PusC[4] + PusNC[4] + PusC[5] + PusNC[5] + PusC[6] + PusNC[6] + PusC[7] + PusNC[7] + PusC[8] + PusNC[8] + PusC[9] + PusNC[9] + PusC[10] + PusNC[10] + PusC[11] + PusNC[11] + PusC[12] + PusNC[12] + PusC[13] + PusNC[13] + PusC[14] + PusNC[14] + PusC[15] + PusNC[15] + PusC[16] + PusNC[16] + PusC[17] + PusNC[17] + PusC[18] + PusNC[18] + PusC[19] + PusNC[19] + PusC[20] + PusNC[20] + PusC[21] + PusNC[21] + PusC[22] + PusNC[22] + PusC[23] + PusNC[23] + PusC[24] + PusNC[24] ≤ 252.1527

In [78]:
#Critical NonCritical Energy
#basically, we are letting the program decide at each hour how much uncritical demand to serve and how much critical demand to serve
#Add in some constraints
for t = 1:T
    @addConstraint(m, PlC[t] + PlNC[t] == Pl[t])
end 

for t = 1:T
    @addConstraint(m, PusC[t] <= PlC[t])
    @addConstraint(m, PusNC[t] <= PlNC[t])   
end 

In [79]:
#DIESELEFFtable_alpha = readcsv("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/dieselEFFTable_alpha.csv")
#DIESELEFFtable_beta = readcsv("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/dieselEFFTable_beta.csv")
DIESELEFFtable_alpha = readcsv("dieselEFFTable_alpha.csv")
DIESELEFFtable_beta = readcsv("dieselEFFTable_beta.csv")

#alpha and betas are values derived from the slope and y intercept of linear approximations 
#of diesel use (l/hour) vs. Load fraction of generator 
#Each generator has its own set of alphas and betas

ALPHAv = DIESELEFFtable_alpha[Xmax+1,1:3]#[0.454, 0.050, 0.100]
BETAv = DIESELEFFtable_beta[Xmax+1,1:3]#[0.568, 0.366, 0.404]

#Insert the diesel generator stuff below
Nv = 3
@defVar(m, Xv[t=1:T,v=1:Nv] >=0)
@defVar(m, Bdv[t=1:T,v=1:Nv], Bin) #diesel on or off

LOADFRACmin = [0.25, 0.5, 0.75]
LOADFRACmax = [0.5, 0.75, 1]
#Xmax  is read in (max of the diesel generator)

#Linear piecewise approx of generator efficiency effects on generator fuel usage

if Xmax > 0


    for t = 1:T
        @addConstraint(m, delta*sum{ALPHAv[v]*Xv[t,v]/Xmax + BETAv[v]*Bdv[t,v], v=1:Nv} == q[t]) #need to reformulate so its linear
        @addConstraint(m, sum{Bdv[t,v], v=1:Nv} <=1)
        @addConstraint(m, sum{Xv[t,v], v=1:Nv} == Pd[t])
   
    end 

    for  t = 1:T
        for  v = 1:Nv 
            @addConstraint(m, sum{Xv[t,v], v=1:Nv}/Xmax <= LOADFRACmax[v]*Bdv[t,v])
            @addConstraint(m, sum{Xv[t,v], v=1:Nv}/Xmax >= LOADFRACmin[v]*Bdv[t,v])
        end 
    end 

    for t = 1:T 
        @addConstraint(m, Cc*q[t] == C[t]) 
    end 

    Bdv0 = 0
    for t=1 
        #for v=1
        @addConstraint(m, Cd*sum{Bdv[t,v] - Bdv0, v=1:Nv} <= Cs[t]) #equation for t =1 only
        #end 
    end 

    for t = 2:T #for T > 1
        @addConstraint(m, Cd*sum{Bdv[t,v] - Bdv[t-1,v], v= 1:Nv} <= Cs[t])
    end 
    
else
    for t=1:T
        @addConstraint(m, Pd[t] == 0)
    end 
end 


In [80]:
@setObjective(m, Min, delta*sum{C[t], t = 1:T} + sum{Cs[t], t=1:T} + CusC *delta*sum{PusC[t], t=1:T} + CusNC *delta*sum{PusNC[t], t=1:T} + Clost*delta*sum{Plost[t], t=1:T} + Cb*sum{SOH[t], t=1:T}) 

C[1] + C[2] + C[3] + C[4] + C[5] + C[6] + C[7] + C[8] + C[9] + C[10] + C[11] + C[12] + C[13] + C[14] + C[15] + C[16] + C[17] + C[18] + C[19] + C[20] + C[21] + C[22] + C[23] + C[24] + Cs[1] + Cs[2] + Cs[3] + Cs[4] + Cs[5] + Cs[6] + Cs[7] + Cs[8] + Cs[9] + Cs[10] + Cs[11] + Cs[12] + Cs[13] + Cs[14] + Cs[15] + Cs[16] + Cs[17] + Cs[18] + Cs[19] + Cs[20] + Cs[21] + Cs[22] + Cs[23] + Cs[24] + PusC[1] + PusC[2] + PusC[3] + PusC[4] + PusC[5] + PusC[6] + PusC[7] + PusC[8] + PusC[9] + PusC[10] + PusC[11] + PusC[12] + PusC[13] + PusC[14] + PusC[15] + PusC[16] + PusC[17] + PusC[18] + PusC[19] + PusC[20] + PusC[21] + PusC[22] + PusC[23] + PusC[24] + 0.33 PusNC[1] + 0.33 PusNC[2] + 0.33 PusNC[3] + 0.33 PusNC[4] + 0.33 PusNC[5] + 0.33 PusNC[6] + 0.33 PusNC[7] + 0.33 PusNC[8] + 0.33 PusNC[9] + 0.33 PusNC[10] + 0.33 PusNC[11] + 0.33 PusNC[12] + 0.33 PusNC[13] + 0.33 PusNC[14] + 0.33 PusNC[15] + 0.33 PusNC[16] + 0.33 PusNC[17] + 0.33 PusNC[18] + 0.33 PusNC[19] + 0.33 PusNC[20] + 0.33 PusNC[21] + 0.33 Pu

In [81]:
#Energy Balance
#Pd = total diesel (for demand and for charging battery)
#Pdb = total diesel for charging battery
#Ppv = total available solar
#Ppvbc = solar used to charge the battery
#Plost = spilled solar
#Pbd = battery discharge
@defVar(m, Ppvbc[1:T] >= 0)
@defVar(m, Pdbc[1:T] >= 0)

#@defVar(m, Ppvmg[1:T] >= 0)

for t = 1:T
    #This equation refers to the amount of energy used to meet demand
	#Here, Pd is the total deisel used but Pd - Pdbc is the diesel applied to meet demand
    @addConstraint(m, (Pd[t]-Pdbc[t]) + Pus[t] + (Ppv[t]*PVbank-Ppvbc[t]-Plost[t]) +Pbd[t] == Pl[t]) #should try to fill Pbc first with available resources, then if there is too much for the
    @addConstraint(m, Ppv[t]*PVbank >= Ppvbc[t]+Plost[t])
    @addConstraint(m, Pd[t] >= Pdbc[t])
	@addConstraint(m, Pdbc[t] + Ppvbc[t]== Pbc[t])
    
    #combined Pdbc[t] and Ppvbc[t]
    #@addConstraint(m, (Pd[t]) + Pus[t] + (Ppv[t]-Plost[t]) + Pbd[t] -Pbc[t] == Pl[t])
    #@addConstraint(m, Ppv[t] +Pd[t] >= Pbc[t]+ Plost[t])
	#@addConstraint(m, Ppv[t] >= Plost[t])
end

In [82]:
#ALPHAbc = [-0 -0 -0.02]#[-0 -0 -0.02 -0.17 -1.43 -6.07]
#BETAbc = [1 1 1.01]#[1 1 1 1.09 1.98 6.14]

#ALPHAbd = [6.07 1.43 0.17]#[6.07 1.43 0.17 0.02 0 0]
#BETAbd = [0.07 0.56 0.91]#[0.07 0.56 0.91 0.99 1 1]

#LIMITS = [0 0.25 0.5 0.75 1]#[0 0.1 0.3 0.5 0.7 0.9 1]

####ANOTHER TEST SET
#ALPHAbc = [-0 -0.2 -6.1]
#BETAbc = [1 1.1 6.14]

#ALPHAbd = [6.1 0 0]
#BETAbd = [0.1 1 1]

#LIMITS = [0 0.5 0.9 1]

###A THIRD TEST SET
ALPHAbc = [0 0 -0.2 -1]
BETAbc = [1 1 1.1 1.7]

ALPHAbd = [3.4 0.3 0 0]
BETAbd = [0.1 0.9 1 1]

#LIMITS = [0 0.25 0.5 0.75 1]

NvB = 4
#@defVar(m, Bbatt[t=1:T,vB=1:NvB], Bin)

#for t=1:T
#@addConstraint(m, sum{Bbatt[t,vB], vB=1:NvB} == 1)
#end 

#for t=1:T
#    @addConstraint(m, SOC[t] <=  sum{LIMITS[vB]*Bbatt[t,vB], vB=2:NvB}) #Upper limit of segment
#    @addConstraint(m, SOC[t] >=  sum{LIMITS[vB]*Bbatt[t,vB], vB=1:(NvB-1)}) #Lower limit of segment
    
#end 

for t=1:T
	for vB = 1:NvB
	@addConstraint(m, Pb_dc*(BETAbc[vB]+ALPHAbc[vB]*SOC[t]) >= Pbc[t]) #Pmax for charge is Pb_dc
	@addConstraint(m, Pb*(BETAbd[vB]+ALPHAbd[vB]*SOC[t]) >= Pbd[t]) #Pmax for discharge is Pb
    
	#@addConstraint(m, Pb*sum{(BETAbc[vB]+ALPHAbc[vB]*(LIMITS[vB]*(-1/2)+LIMITS[vB-1]*3/2))*Bbatt[t,vB], vB=2:NvB} >= Pbc[t]) #Pmax
    #@addConstraint(m, Pb*sum{(BETAbd[vB]+ALPHAbd[vB]*(LIMITS[vB]/2+LIMITS[vB-1]/2))*Bbatt[t,vB], vB=2:NvB} >= Pbd[t]) #Pmax
    
    #@addConstraint(m, Pb*sum{(BETAbc[vB]+ALPHAbc[vB]*(LIMITS[vB]-LIMITS[vB-1]))*Bbatt[t,vB], vB=2:NvB}/2 >= Pbc[t]) #Pmax
    #@addConstraint(m, Pb*sum{(BETAbd[vB]+ALPHAbd[vB]*(LIMITS[vB]-LIMITS[vB-1]))*Bbatt[t,vB], vB=2:NvB}/2 >= Pbd[t]) #Pmax
    #@addConstraint(m, Pb*sum{(BETAbc[vB]+ALPHAbc[vB]*(LIMITS[vB]))*Bbatt[t,vB], vB=2:NvB} >= Pbc[t]) #Pmax
    #@addConstraint(m, Pb*sum{(BETAbd[vB]+ALPHAbd[vB]*(LIMITS[vB]))*Bbatt[t,vB], vB=2:NvB} >= Pbd[t]) #Pmax
    
    #@addConstraint(m, Pb*sum{(BETAbc[vB]+ALPHAbc[vB]*(LIMITS[vB-1]))*Bbatt[t,vB], vB=2:NvB} <= Pbc[t]) #Pmin- don't think this makes snse
    #@addConstraint(m, Pb*sum{(BETAbd[vB]+ALPHAbd[vB]*(LIMITS[vB-1]))*Bbatt[t,vB], vB=2:NvB} <= Pbd[t]) #Pmin
    end
end

@addConstraint(m, SOC[1] == 0.1)
for t = 1:T
    @addConstraint(m, SOC[t] <= 0.9)
    @addConstraint(m, SOC[t] >= 0.1)
end 

if Eb > 0
    for t = 2:T  
        @addConstraint(m, SOC[t-1] - (Pbd[t-1]/(EFFd*Eb) - (EFFc/Eb)*Pbc[t-1]) == SOC[t])
    
        end 
else 
    for t = 2:T
        @addConstraint(m, SOC[t] == SOC[1])
    end 
end 



In [83]:
for t = 1:T
    @addConstraint(m, Pb_dc*Bbc[t] >= Pbc[t])
    @addConstraint(m, Pb*Bbd[t] >= Pbd[t])
    @addConstraint(m, Bbc[t] + Bbd[t] <= 1)
end 


In [84]:
cycles = 100
for t=1:T
    @addConstraint(m, 0.5*(delta/cycles*Eb*Pbd[t]) + 0.5*(delta/cycles*Eb*Pbc[t]) == SOH[t])
end 

In [85]:
solve(m)

Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01



:Infeasible

In [86]:
getObjectiveValue(m)

1.7976931348623157e308

In [87]:
totalUnserved = 0
totalDemand = 0

for t = 1:T
    totalUnserved = totalUnserved + getValue(PusC[t]) + getValue(PusNC[t])
    totalDemand = totalDemand + getValue(Pl[t])
end 

totalUnserved/totalDemand

0.6

In [88]:
totalUnserved

252.15270000000004

In [89]:
totalDemand

420.25450000000006

In [90]:
#Printing results to text files

dieselCost = 0
genStartUpCost = 0
unservedCritCost = 0
unservedNCritCost = 0
spillCost = 0
battDegradeCost = 0 

#getting the sums first 
for t = 1:T
    dieselCost = dieselCost + getValue(C[t]) #have to use getValue or it will just output the variable
    genStartUpCost = genStartUpCost + getValue(Cs[t])
    unservedCritCost = unservedCritCost + getValue(PusC[t])
    unservedNCritCost =unservedNCritCost + getValue(PusNC[t])
    spillCost = spillCost + getValue(Plost[t])
    battDegradeCost = battDegradeCost + getValue(SOH[t])
end 

#multiplying to get the actual values 
dieselCost = delta*dieselCost
genStartUpCost = genStartUpCost
unservedCritCost = CusC*delta*unservedCritCost
unservedNCritCost = CusNC*delta*unservedNCritCost
spillCost = Clost*delta*spillCost
battDegradeCost = Cb*battDegradeCost

###### print dispatch


g = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/Diesel Use.txt", "w")
writedlm(g, getValue(q))
close(g)

#SOC
f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/SOC.txt", "w")
writedlm(f, getValue(SOC))
close(f)



#Unserved Energy Dispatch
f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/PusC Dispatch.txt", "w")
writedlm(f, getValue(PusC))
close(f)

f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/PusNC Dispatch.txt", "w")
writedlm(f, getValue(PusNC))
close(f)

#Solar Related Dispatch
f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/Plost Dispatch.txt", "w")
writedlm(f, getValue(Plost))
close(f)

f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/Ppv Dispatch.txt", "w")
#writedlm(f, getValue(Ppv*PVbank))
writedlm(f, Ppv*PVbank)
close(f)


#battery dispatch
f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/Pbd Dispatch.txt", "w")
writedlm(f, getValue(Pbd))
close(f)

#battery charge
f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/Pdbc Dispatch.txt", "w") #battery charge total
writedlm(f, getValue(Pdbc))
close(f)

f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/Ppvbc Dispatch.txt", "w") #battery charge total
writedlm(f, getValue(Ppvbc))
close(f)

#Diesel dispatch

f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/Pd Dispatch.txt", "w")
writedlm(f, getValue(Pd))
close(f)

#Printing total demand

f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/demandData.txt", "w")
writedlm(f, getValue(Pl))
close(f)

#####print costs
costs = [dieselCost genStartUpCost unservedCritCost unservedNCritCost spillCost battDegradeCost]

f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/costs.txt", "w")
writedlm(f, costs)
close(f)


#Doesn't work right now
#f = open("C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/PvData.txt", "w")
#writedlm(f, Ppv)
#close(f)














LoadError: opening file C:/Users/V/AppData/Local/Julia-0.3.8/dispatch/Diesel Use.txt: No such file or directory
while loading In[90], in expression starting on line 31